In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.38.1
Uninstalling transformers-4.38.1:
  Successfully uninstalled transformers-4.38.1
Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2
  Using cached transformers-4.38.1-py3-none-any.whl (8.5 MB)
  Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)


In [3]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [5]:
# Load the 'K45lim/resume_classifier' dataset
dataset = load_dataset("K45lim/resume_classifierV1")
dataset


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['Resume', 'Category'],
        num_rows: 1738
    })
    validation: Dataset({
        features: ['Resume', 'Category'],
        num_rows: 246
    })
    test: Dataset({
        features: ['Resume', 'Category'],
        num_rows: 499
    })
})

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [7]:
model_ckpt = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_berto = AutoModelForSequenceClassification.from_pretrained(model_ckpt).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Resume', 'Category'],
        num_rows: 1738
    })
    validation: Dataset({
        features: ['Resume', 'Category'],
        num_rows: 246
    })
    test: Dataset({
        features: ['Resume', 'Category'],
        num_rows: 499
    })
})

In [9]:
# Tokenize input text data
def tokenize_data(example):
#    return tokenizer(example['Resume'], padding='max_length', truncation=True), example['Category']
    return tokenizer(example['Resume'], truncation=True), example['Category']

def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['Resume'] , padding=True , truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['Category'], padding=True, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }


In [10]:
tokenized_dataset = dataset.map(convert_examples_to_features, batched = True)

In [11]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['Resume', 'Category', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1738
    })
    validation: Dataset({
        features: ['Resume', 'Category', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 246
    })
    test: Dataset({
        features: ['Resume', 'Category', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 499
    })
})

In [12]:
tokenized_dataset['train']['Resume'][1]

'DATABASE PROGRAMMER/ANALYST (.NET DEVELOPER)       Summary     Software Engineer with 2 years in Web Developer specializing in front end development. Proficient in Software Development Life Cycle (SDLC) and SCRUM AGILE methodologies of development process to produce software solutions by team. Involved in designing, planning, developing and implementing Microsoft .Net based solutions that meet quality and code standards.  Developed web based applications using C#, ASP.Net, JavaScript, jQuery, CSS, Bootstrap CSS, HTML, XML and AJAX. Strong experience in data retrieval and manipulation using ADO.Net data objects such as Dataset, Data Reader, Data list, Data Adaptor, Entity Frame work. Good experience in writing Class Library using C# , LINQ to SQL queries in Database Access layer to interface with SQL Database. Worked extensively with .Net Server Controls, Web User Controls, Data Grid, Web Control, Form Validation Controls and created Custom controls. Worked on creating and modifying Ta

In [13]:
tokenized_dataset['train']['Category'][1]

'INFORMATION-TECHNOLOGY'

In [14]:
tokenized_dataset['train']['labels'][1]

[101, 2592, 1011, 2974, 102]

In [15]:
# Prepare for the labels

In [16]:
job_position = sorted(list(set(tokenized_dataset['train']["Category"])))
job_position

['ACCOUNTANT',
 'ADVOCATE',
 'AGRICULTURE',
 'APPAREL',
 'ARTS',
 'AUTOMOBILE',
 'AVIATION',
 'BANKING',
 'BPO',
 'BUSINESS-DEVELOPMENT',
 'CHEF',
 'CONSTRUCTION',
 'CONSULTANT',
 'DESIGNER',
 'DIGITAL-MEDIA',
 'ENGINEERING',
 'FINANCE',
 'FITNESS',
 'HEALTHCARE',
 'HR',
 'INFORMATION-TECHNOLOGY',
 'PUBLIC-RELATIONS',
 'SALES',
 'TEACHER']

In [17]:
label2id = {role: i for i, role in enumerate(job_position)}
print(label2id)

{'ACCOUNTANT': 0, 'ADVOCATE': 1, 'AGRICULTURE': 2, 'APPAREL': 3, 'ARTS': 4, 'AUTOMOBILE': 5, 'AVIATION': 6, 'BANKING': 7, 'BPO': 8, 'BUSINESS-DEVELOPMENT': 9, 'CHEF': 10, 'CONSTRUCTION': 11, 'CONSULTANT': 12, 'DESIGNER': 13, 'DIGITAL-MEDIA': 14, 'ENGINEERING': 15, 'FINANCE': 16, 'FITNESS': 17, 'HEALTHCARE': 18, 'HR': 19, 'INFORMATION-TECHNOLOGY': 20, 'PUBLIC-RELATIONS': 21, 'SALES': 22, 'TEACHER': 23}


In [18]:
id2label = {i: role for i, role in enumerate(job_position)}
print(id2label)

{0: 'ACCOUNTANT', 1: 'ADVOCATE', 2: 'AGRICULTURE', 3: 'APPAREL', 4: 'ARTS', 5: 'AUTOMOBILE', 6: 'AVIATION', 7: 'BANKING', 8: 'BPO', 9: 'BUSINESS-DEVELOPMENT', 10: 'CHEF', 11: 'CONSTRUCTION', 12: 'CONSULTANT', 13: 'DESIGNER', 14: 'DIGITAL-MEDIA', 15: 'ENGINEERING', 16: 'FINANCE', 17: 'FITNESS', 18: 'HEALTHCARE', 19: 'HR', 20: 'INFORMATION-TECHNOLOGY', 21: 'PUBLIC-RELATIONS', 22: 'SALES', 23: 'TEACHER'}


In [19]:
# Setup the Trainer

In [31]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=24, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [33]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer, return_tensors="tf")

In [36]:
generation_data_collator = DataCollatorWithPadding(tokenizer, return_tensors="tf")

In [38]:
trainer_args = TrainingArguments(
    output_dir='new_model', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [39]:

trainer = Trainer(model=model_berto, args=trainer_args,
                  tokenizer=tokenizer, data_collator=data_collator,
                  train_dataset=tokenized_dataset["test"],
                  eval_dataset=tokenized_dataset["validation"])


In [40]:
trainer.train()

AttributeError: EagerTensor object has no attribute 'size'. 
        If you are looking for numpy-related methods, please run the following:
        tf.experimental.numpy.experimental_enable_numpy_behavior()
      